# Why this notebook
* Each model is different in how the CONF parameter decides PRECISION and RECALL
* Finding which CONF level is often a matter of guesswork, and you are limited by 5 submissions per day.
* By systematically evaluating F2 score at each level of CONF, you can have a good idea which CONF will give you the best competition F2 metric.

* This assumes:
  * Your validation dataset is similar to the hidden test set
  * Your model has not seen your validation dataset

### What's new in this notebook?
* F2 is evaluated at every CONF level on validation dataset, similar to F1_curve.png for YoloV5
* Define 'eval_IOU' for which IOU level to evaluate F2 at
  * (Competition metric tests IOU @ 0.3 to 0.8 with step of 0.05)

### Example model used in this notebook

* This notebook uses the YOLOX model kindly provided by Remek Kinas (https://www.kaggle.com/remekkinas/yolox-inference-on-kaggle-for-cots-lb-0-507)
  * And assumes it is trained on FOLD=4 of 5-fold split performed in: https://www.kaggle.com/remekkinas/yolox-training-pipeline-cots-dataset-lb-0-507

### Warning
* Note the iteration time as an estimate to how long this notebook takes to run.
* Minimise wasting precious GPU minutes running at large resolutions. Test at small resolutions! The CONF peak should theoretically be the same.

# Control Panel (change your settings here)

In [ ]:
# CONTROL PANEL

MODELPATH = "yolox-cots-models"

MODELNAME = "yx_l_003"

MODEL_TYPE = "L" # YOLOX "S", M", "L" or "X"

# YOLOX Inference Size (h, w) - each must be multiple of 32
SIZE = (800, 1280)

# YOLOX NMS
NMS_THRESHOLD = 0.4

# Which IOU level to evaluate (Competition metric tests 0.3 to 0.8 with step of 0.05)
eval_IOU = 0.65

### YOLOX Inference Model

In [ ]:
CHECKPOINT_FILE = f"/kaggle/input/{MODELPATH}/{MODELNAME}.pth"

# Installation and Code

## Import Modules

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import torch
import importlib
import cv2 
import pandas as pd
import numpy as np

import ast
import shutil
import sys

from tqdm.notebook import tqdm
tqdm.pandas()

from PIL import Image
from IPython.display import display

## YOLOX Installation

* This notebook only installs the minimum required for YOLOX inference. Each GPU minute is precious!

In [ ]:
# Copy YOLOX and required modules from local repository (Kaggle dataset -> https://www.kaggle.com/remekkinas/yolox-cots-models)
%cp -r /kaggle/input/yolox-cots-models /kaggle/working/

In [ ]:
# Install YOLOX required modules (those already installed by default Kaggle are commented out)
%cd /kaggle/working/yolox-cots-models/yolox-dep

# !pip install pip-21.3.1-py3-none-any.whl -f ./ --no-index
!pip install loguru-0.5.3-py3-none-any.whl -f ./ --no-index
!pip install ninja-1.10.2.3-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl -f ./ --no-index
# !pip install onnx-1.8.1-cp37-cp37m-manylinux2010_x86_64.whl -f ./ --no-index
# !pip install onnxruntime-1.8.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl -f ./ --no-index
# !pip install onnxoptimizer-0.2.6-cp37-cp37m-manylinux2014_x86_64.whl -f ./ --no-index
!pip install thop-0.0.31.post2005241907-py3-none-any.whl -f ./ --no-index
# !pip install tabulate-0.8.9-py3-none-any.whl -f ./ --no-index
#!pip install onnx-simplifier-0.3.6.tar.gz -f ./ --no-index

In [ ]:
# Install YOLOX (this step is actually not necessary for inference)
# %cd /kaggle/working/yolox-cots-models/YOLOX
# !pip install -r requirements.txt
# !pip install -v -e . 

In [ ]:
# Install CocoAPI tool
%cd /kaggle/working/yolox-cots-models/yolox-dep/cocoapi/PythonAPI

!make
!make install
!python setup.py install

In [ ]:
import pycocotools

### YOLOX Model and Functions

In [ ]:
%cd /kaggle/working/yolox-cots-models/YOLOX

from string import Template

config_file_template = '''

#!/usr/bin/env python3
# -*- coding:utf-8 -*-
# Copyright (c) Megvii, Inc. and its affiliates.

import os

from yolox.exp import Exp as MyExp


class Exp(MyExp):
    def __init__(self):
        super(Exp, self).__init__()
        self.depth = $model_depth
        self.width = $model_width
        self.exp_name = os.path.split(os.path.realpath(__file__))[1].split(".")[0]
        self.num_classes = 1

'''
if MODEL_TYPE == "X":
    pipeline = Template(config_file_template).substitute(
        model_depth = 1.33,
        model_width = 1.25
    )
elif MODEL_TYPE == "L":
    pipeline = Template(config_file_template).substitute(
        model_depth = 1.0,
        model_width = 1.0
    )
elif MODEL_TYPE == "M":
    pipeline = Template(config_file_template).substitute(
        model_depth = 0.67,
        model_width = 0.75
    )
elif MODEL_TYPE == "S":
    pipeline = Template(config_file_template).substitute(
        model_depth = 0.67,
        model_width = 0.75
    )

with open('cots_config.py', 'w') as f:
    f.write(pipeline)

In [ ]:
%cd /kaggle/working/yolox-cots-models/YOLOX

from yolox.utils import postprocess
from yolox.data.data_augment import ValTransform

COCO_CLASSES = (
  "starfish",
)

# get YOLOX experiment
current_exp = importlib.import_module('cots_config')
exp = current_exp.Exp()

num_classes = 1

nmsthre = NMS_THRESHOLD

# get YOLOX model
model = exp.get_model()
model.cuda()
model.eval()

# get custom trained checkpoint
ckpt_file = CHECKPOINT_FILE
ckpt = torch.load(ckpt_file, map_location="cpu")
model.load_state_dict(ckpt["model"])

In [ ]:
# Modified from https://www.kaggle.com/remekkinas/yolox-inference-on-kaggle-for-cots-lb-0-507
# Additions: 
#     auto converts to xywh format
#     converts tensors to list of floats

def yolox_inference(img, model, test_size, conf_threshold = 0.4):
    bboxes = []
    bbclasses = []
    scores = []
    
    preproc = ValTransform(legacy = False)
    
    tensor_img, _ = preproc(img, None, test_size)
    tensor_img = torch.from_numpy(tensor_img).unsqueeze(0)
    tensor_img = tensor_img.float()
    tensor_img = tensor_img.cuda()
    
    with torch.no_grad():
        outputs = model(tensor_img)
        outputs = postprocess(
                    outputs, num_classes, conf_threshold,
                    nmsthre, class_agnostic=True
                )
    
    if outputs[0] is None:
        return [], [], []
    
    outputs = outputs[0].cpu()
    bboxes = outputs[:, 0:4]
    
    bboxes /= min(test_size[0] / img.shape[0], test_size[1] / img.shape[1])
    bbclasses = outputs[:, 6]
    scores = outputs[:, 4] * outputs[:, 5]
    
    if len(bboxes) == 0:
        return [], [], []
    
    bboxes = bboxes.numpy()
    
    # format to coco
    bboxes[:, 2] = bboxes[:, 2] - bboxes[:, 0]
    bboxes[:, 3] = bboxes[:, 3] - bboxes[:, 1]    
    
    # Converts tensors to lists
    return bboxes, bbclasses.tolist(), scores.tolist()

In [ ]:
# Modified from https://www.kaggle.com/remekkinas/yolox-inference-on-kaggle-for-cots-lb-0-507
# Additions: 
#     allows customized box color (BGR)

def draw_yolox_predictions(img, bboxes, scores, bbclasses, classes_dict, boxcolor = (0,0,255)):
    outimg = img.copy()
    for i in range(len(bboxes)):
        box = bboxes[i]
        cls_id = int(bbclasses[i])
        score = scores[i]
        x0 = int(box[0])
        y0 = int(box[1])
        x1 = x0 + int(box[2])
        y1 = y0 + int(box[3])

        cv2.rectangle(outimg, (x0, y0), (x1, y1), boxcolor, 2)
        cv2.putText(outimg, '{}:{:.1f}%'.format(classes_dict[cls_id], score * 100), (x0, y0 - 3), cv2.FONT_HERSHEY_PLAIN, 0.8, boxcolor, thickness = 1)
    return outimg

## IOU Calculation

In [ ]:
def IOU_coco(bbox1, bbox2):
    '''
        adapted from https://stackoverflow.com/questions/25349178/calculating-percentage-of-bounding-box-overlap-for-image-detector-evaluation
    '''
    x_left = max(bbox1[0], bbox2[0])
    y_top = max(bbox1[1], bbox2[1])
    x_right = min(bbox1[0] + bbox1[2], bbox2[0] + bbox2[2])
    y_bottom = min(bbox1[1] + bbox1[3], bbox2[1] + bbox2[3])
    if x_right < x_left or y_bottom < y_top:
        return 0.0
    intersection_area = (x_right - x_left) * (y_bottom - y_top)
    bb1_area = bbox1[2] * bbox1[3]
    bb2_area = bbox2[2] * bbox2[3]
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)

    assert iou >= 0.0
    assert iou <= 1.0
    return iou

# Retrieve video frames not used in training

In [ ]:
%cd /kaggle/working

from sklearn.model_selection import GroupKFold

def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_path(row):
    row['image_path'] = f'{ROOT_DIR}/train_images/video_{row.video_id}/{row.video_frame}.jpg'
    return row

ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'

df = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/train.csv")

# Don't filter for annotated frames. Include frames with no bboxes as well!
df["num_bbox"] = df['annotations'].apply(lambda x: str.count(x, 'x'))
df_train = df

# Annotations 
df_train['annotations'] = df_train['annotations'].progress_apply(lambda x: ast.literal_eval(x))
df_train['bboxes'] = df_train.annotations.progress_apply(get_bbox)

df_train = df_train.progress_apply(get_path, axis=1)

kf = GroupKFold(n_splits = 5) 
df_train = df_train.reset_index(drop=True)
df_train['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(kf.split(df_train, y = df_train.video_id.tolist(), groups=df_train.sequence)):
    df_train.loc[val_idx, 'fold'] = fold

df_train.head(5)

### Select the dataset that your model hasn't seen!

In [ ]:
df_test = df_train[df_train.fold == 4]

### Get image paths and ground truth BB's

In [ ]:
import copy

# deepcopy is required to avoid 

df_sample = df_test
image_paths = df_sample.image_path.tolist()
gt = copy.deepcopy(df_sample.bboxes.tolist())
gtmem = copy.deepcopy(df_sample.bboxes.tolist())

# Test your model is working

In [ ]:
i = 1350
TEST_IMAGE_PATH = image_paths[i]
img = cv2.imread(TEST_IMAGE_PATH)

bboxes, bbclasses, scores = yolox_inference(img, model, SIZE, 0.01)

# Draw Green ground truth box
out_image = draw_yolox_predictions(img, gt[i], [1.0] * len(gt[i]), [0] * len(gt[i]), COCO_CLASSES, (0,255,0))

# Draw Red inference box
out_image = draw_yolox_predictions(out_image, bboxes, scores, bbclasses, COCO_CLASSES, (0,0,255))

# Convert BGR to RGB
out_image = cv2.cvtColor(out_image, cv2.COLOR_BGR2RGB)

display(Image.fromarray(out_image))

# Assess Model Performance

In [ ]:
%cd /kaggle/working

# Confidence scores of true positives, false positives and count false negatives
TP = [] # Confidence scores of true positives
FP = [] # Confidence scores of true positives
FN = 0  # Count of false negative boxes

for i in tqdm(range(len(image_paths))):
# for i in tqdm(range(1250, 1450)):
    TEST_IMAGE_PATH = image_paths[i]
    img = cv2.imread(TEST_IMAGE_PATH)
    bboxes, bbclasses, scores = yolox_inference(img, model, SIZE, 0.01)

    # Test YOLOX
    gt0 = gt[i]
    if len(bboxes) == 0:
        # all gt are false negative
        FN += len(gt0)
    else:
        bb = bboxes.copy().tolist()
        for idx, b in enumerate(bb):
            b.append(scores[idx])
        bb.sort(key = lambda x: x[4], reverse = True)
        
        if len(gt0) == 0:
            # all bboxes are false positives
            for b in bb:
                FP.append(b[4])
        else:
            # match bbox with gt
            for b in bb:
                matched = False
                for g in gt0:
                    # check whether gt box is already matched to an inference bb
                    if len(g) == 4:
                        # g bbox is unmatched
                        if IOU_coco(b, g) >= eval_IOU:
                            g.append(b[4]) # assign confidence values to g; marks g as matched
                            matched = True
                            TP.append(b[4])
                            break
                if not matched:
                    FP.append(b[4])
            for g in gt0:
                if len(g) == 4:
                    FN += 1

# Display your model's Metrics

In [ ]:
from matplotlib import pyplot as plt

plt.hist(TP, 100)
plt.title("CONF of true positives, base YOLOX")
plt.xlabel('CONF')
plt.ylabel('TP count')
plt.show()

print(f'True positives = {len(TP)}')
print(f'False negatives = {FN}')

In [ ]:
plt.hist(FP, 100)
plt.title("CONF of false positives, base YOLOX")
plt.xlabel('CONF')
plt.ylabel('FP count')
plt.show()

print(f'False positives = {len(FP)}')

In [ ]:
F2list = []
F2max = 0.0
F2maxat = -1.0

for c in np.arange(0.0, 1.0, 0.01):
    FNcount = FN + sum(1 for i in TP if i < c)
    TPcount = sum(1 for i in TP if i >= c)
    FPcount = sum(1 for i in FP if i >= c)
    R = TPcount / (TPcount + FNcount + 0.0001)
    P = TPcount / (TPcount + FPcount + 0.0001)
    F2 = (5 * P * R) / (4 * P + R + 0.0001)
    F2list.append((c, F2))
    if F2max < F2:
        F2max = F2
        F2maxat = c

plt.scatter(*zip(*F2list))
plt.title("CONF vs F2 score")
plt.xlabel('CONF')
plt.ylabel('F2')
plt.show()

print(f'F2 max is {F2max} at CONF = {F2maxat}')

### Cleanup

In [ ]:
!rm -rf /kaggle/working/*

# Credits

* https://www.kaggle.com/remekkinas/yolox-training-pipeline-cots-dataset-lb-0-507
* https://www.kaggle.com/remekkinas/yolox-inference-on-kaggle-for-cots-lb-0-507